In [15]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [21]:
import pandas
import yfinance
from xgboost import XGBRegressor
from sklearn.impute import KNNImputer

In [17]:
stock_info=pandas.read_csv('Stock_Dataset.csv')
imputer=KNNImputer(n_neighbors=5)
stock_info=pandas.DataFrame(data=imputer.fit_transform(stock_info),columns=['Return_On_Equity','Current_Ratio','Asster_Turnover','WACC','Dividend_Yield','Earnings_Yield','Revenue_Change','Net_Income_Change','Price_Change'])
price_change=stock_info.pop('Price_Change')

In [18]:
xgbooster=XGBRegressor(n_estimators=1000,max_depth=7,eta=.001,subsample=.75,colsample_bytree=.9,num_parallel_tree=1)
xgbooster.fit(stock_info,price_change)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.001, eval_metric=None,
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=None, num_parallel_tree=1, ...)

In [64]:
def get_roe(balance,income):
    if len(income)==0 or len(balance)==0:
        return None
    else:
        try:
            equity=balance['Stockholders Equity'][0]
        except:
            return 0
        else:
            equity=balance['Stockholders Equity'][0]
        net=income['Net Income'][0]
        return net/equity if equity!=0 else 0

def get_current(balance):
    if len(balance)==0:
        return None
    else:
        
        try:
            c_asset=balance['Current Assets'][0]
        except:
            c_asset=0
        else:
            c_asset=balance['Current Assets'][0] 
            
        try:
            c_liab=balance['Current Liabilities'][0]
        except:
            try:
                c_liab=balance['Current Debt'][0]
            except:
                c_liab=0
            else:
                c_liab=balance['Current Debt'][0]
        else:
            c_liab=balance['Current Liabilities'][0]
        return c_asset/c_liab if c_liab!=0 else 1
    
def get_turnover(balance,income):
    if len(balance)==0:
        return None
    else:
        
        try:
            sales=income['Total Revenue'][0]
        except:
            return 1
        else:
            sales=income['Total Revenue'][0]
        try:    
            assets=balance['Total Assets'][0]
        except:
            assets=0
        return sales/assets if assets!=0 else None

def get_wacc(balance,income,beta):
    if len(income)==0:
        return None
    else:
        risk_free_rate=.0438
        market_rate=.093
        
        try:
            tax_rate=income['Tax Provision'][0]/income['Pretax Income'][0]
        except:
            tax_rate=0
        else:
            tax_rate=income['Tax Provision'][0]/income['Pretax Income'][0]
        
        
        try:
            cap=balance['Total Capitalization'][0]
        except:
            cap=1
        else:
            cap=balance['Total Capitalization'][0]
        
        try:
            liab=balance['Total Debt'][0]
        except:
            liab=1
        else:
            liab=balance['Total Debt'][0]
        w_e=cap/(cap+liab)
        w_d=liab/(cap+liab)
        
        try:
            interest=income['Interest Expense'][0]
        except:
            interest=0
        else:
            interest=income['Interest Expense'][0]
        
        try:
            avg_liab=(balance['Total Debt'][0]+balance['Total Debt'][1])/2
        except:
            avg_liab=0
        else:
            avg_liab=(balance['Total Debt'][0]+balance['Total Debt'][1])/2
            
        k_e=risk_free_rate+beta*(market_rate-risk_free_rate)
        k_d=(interest/avg_liab)*(1-tax_rate) if avg_liab!=0 else 1
        return w_e*k_e+w_d*k_d
    
def get_div(income,cashflow,price):
    if len(price)==0:
        return None
    else:
        try:
            div=cashflow['Cash Dividends Paid'][0]
        except:
            return 0
        else:
            div=cashflow['Cash Dividends Paid'][0]
        try:
            shares=income['Basic Average Shares'][0]
        except:
            shares=0
        else:
            shares=income['Basic Average Shares'][0]

        end_price=price['Close'][-1]
        if shares==0 or end_price==0:
            return 0
        else:
            return ((div*-1)/shares)/end_price
        
def get_earnings(income,price):
    if len(price)==0:
        return None
    else:
        try:
            eps=income['Basic EPS'][0]
        except:
            eps=0
        else:
            eps=income['Basic EPS'][0]
            
        end_price=price['Close'][-1]
        return eps/end_price if end_price!=0 else 0
    
def get_rev(income):
    if len(income)==0:
        return None
    else:
        try:
            income['Total Revenue'][0]/income['Total Revenue'][1]
        except:
            return 1
        else:
            return income['Total Revenue'][0]/income['Total Revenue'][1]
        
def get_ni(income):
    if len(income)==0:
        return None
    else:
        try:
            income['Net Income'][0]/income['Net Income'][1]
        except:
            return 1
        else:
            return income['Net Income'][0]/income['Net Income'][1]

In [90]:
stock_ticker_names=[]
portfolio_weights=[]
while sum(portfolio_weights)<1:
    stock_exists=0
    while not stock_exists:
        stock_choice=input("Please input the stock you want to add to the portfolio.")
        stock_info=yfinance.Ticker(stock_choice).balance_sheet.T
        if not len(stock_info):
            print('Stock not found')
            stock_exists=0
        else:
            stock_exists=1
    stock_ticker_names.append(stock_choice)
    weight_check=1
    while weight_check:
        stock_weight=(float(input("What percentage of the portfolio does this stock take up?")))
        if round(1-float(sum(portfolio_weights))-stock_weight,6)<0:
            print('Sum of portfolio weights exceeds 100%')
        else:
            weight_check=0
    portfolio_weights.append(stock_weight)
    print(str(1-float(sum(portfolio_weights)))+' of the porfolio still needs to be entered')
    print('')

roe=[]
current=[]
turnover=[]
wacc=[]
div=[]
earning=[]
rev=[]
ni=[]

def propogate(company):
    balance=yfinance.Ticker(company).balance_sheet.T
    income=yfinance.Ticker(company).income_stmt.T
    cashflow=yfinance.Ticker(company).cash_flow.T
    price=yfinance.Ticker(company).history(start='2023-01-01',end='2024-01-01')
    try:
        beta=yfinance.Ticker(company).info['beta']
    except:
        beta=1
    else:
        beta=yfinance.Ticker(company).info['beta']
        
    roe.append(get_roe(balance,income))
    current.append(get_current(balance))
    turnover.append(get_turnover(balance,income))
    wacc.append(get_wacc(balance,income,beta))
    div.append(get_div(income,cashflow,price))
    earning.append(get_earnings(income,price))
    rev.append(get_rev(income))
    ni.append(get_ni(income))
    
for stocks in stock_ticker_names:
    propogate(stocks)
    
portfolio_ratio_data=pandas.DataFrame()
portfolio_ratio_data=portfolio_ratio_data.assign(Return_On_Equity=roe)
portfolio_ratio_data=portfolio_ratio_data.assign(Current_Ratio=current)
portfolio_ratio_data=portfolio_ratio_data.assign(Asster_Turnover=turnover)
portfolio_ratio_data=portfolio_ratio_data.assign(WACC=wacc)
portfolio_ratio_data=portfolio_ratio_data.assign(Dividend_Yield=div)
portfolio_ratio_data=portfolio_ratio_data.assign(Earnings_Yield=earning)
portfolio_ratio_data=portfolio_ratio_data.assign(Revenue_Change=rev)
portfolio_ratio_data=portfolio_ratio_data.assign(Net_Income_Change=ni)
portfolio_ratio_data=pandas.DataFrame(data=imputer.fit_transform(portfolio_ratio_data),columns=['Return_On_Equity','Current_Ratio','Asster_Turnover','WACC','Dividend_Yield','Earnings_Yield','Revenue_Change','Net_Income_Change'])

predicted_stock_returns=xgbooster.predict(portfolio_ratio_data.values)
weighted_returns=[]
counter=0
for returns in predicted_stock_returns:
    weighted_returns.append(returns*portfolio_weights[counter])
print('Your portfolio is expected to have a total return of '+str(sum(weighted_returns)*100)+'%.')
print('The model is 95% confident that your portfolio will return in the range of '+str(((sum(weighted_returns)*100-1.96*3.0735),sum(weighted_returns)*100+1.96*3.0735)))

Please input the stock you want to add to the portfolio.TGT
What percentage of the portfolio does this stock take up?.25
0.75 of the porfolio still needs to be entered

Please input the stock you want to add to the portfolio.SCHL
What percentage of the portfolio does this stock take up?.25
0.5 of the porfolio still needs to be entered

Please input the stock you want to add to the portfolio.MCD
What percentage of the portfolio does this stock take up?.3
0.19999999999999996 of the porfolio still needs to be entered

Please input the stock you want to add to the portfolio.KO
What percentage of the portfolio does this stock take up?.2
0.0 of the porfolio still needs to be entered

Your portfolio is expected to have a total return of -0.16416267753811553%.
The model is 95% confident that your portfolio will return in the range of (-6.188222677538116, 5.859897322461885)
